In [5]:
!pip install langchain
!pip install langchain-community
!pip install langchain_google_genai
!pip install --upgrade google-cloud-aiplatform
!pip install chromadb
!pip install langchain-chroma

In [6]:
API_KEY=""

In [7]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Set your Google API Key as an environment variable
os.environ["GOOGLE_API_KEY"] = API_KEY
# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Simple text generation example
result = llm.invoke("Write a motivational quote.")
print(result.content)

"The only limits you have are the ones you set for yourself."


In [8]:
poem = llm.invoke("Write a 4 line poem of my love for samosa")
print(poem.content)

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders import csv_loader

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize instructor embeddings using the Hugging Face model
embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

e = embed.embed_query("What is your refund policy?")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
e[:13]

[0.02429858222603798,
 0.037645600736141205,
 0.006362581625580788,
 0.003087150864303112,
 -0.0393715426325798,
 -0.014242197386920452,
 -0.07340434938669205,
 0.006521736271679401,
 -0.0351148284971714,
 -0.03999587520956993,
 0.060654714703559875,
 -0.04561867564916611,
 0.046161141246557236]

In [12]:
len(e)

768

In [13]:
loader = csv_loader.CSVLoader(file_path="/content/codebasics_faqs.csv",source_column="prompt",encoding="latin-1")
data = loader.load()

In [18]:
vectordb = Chroma.from_documents(documents=data, embedding = embed)
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [19]:
retriever.get_relevant_documents("How about a demo of my application?")

<ipython-input-19-b5259ae88961>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("How about a demo of my application?")


[Document(metadata={'row': 4, 'source': 'I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?'}, page_content='prompt: I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nresponse: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.'),
 Document(metadata={'row': 20, 'source': 'I\x92m not sure if this course is good enough for me to invest some money. What can I do?'}, page_content='prompt: I\x92m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don\x92t worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style'),
 Document(metadata={'row': 

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [23]:
chain.invoke("Is this bootcamp suitable for me if i dont know programming")

{'query': 'Is this bootcamp suitable for me if i dont know programming',
 'result': 'Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.',
 'source_documents': [Document(metadata={'row': 0, 'source': 'I have never done programming in my life. Can I take this bootcamp?'}, page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'),
  Document(metadata={'row': 4, 'source': 'I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?'}, page_content='prompt: I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nre